In [1]:
import sys
tmp = set(sys.path)
tmp.add("/home/work/notebook")
sys.path = list(tmp)
print(sys.path)

['', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python36.zip', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/dataset-1.1.0-py3.6.egg', '/home/work/.ipython', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/IPython/extensions', '/home/work/notebook', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/lib-dynload', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6', '/home/work/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/normality-1.0.0-py3.6.egg']


In [2]:
# GPU setup
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# download datasets
#import urllib.request
#dataset_prefix = 'http://s3.amazonaws.com/cornell-tech-sdl-openrec'
#urllib.request.urlretrieve('%s/lastfm/lastfm_test.npy' % dataset_prefix, 
#                   'lastfm_test.npy')
#urllib.request.urlretrieve('%s/lastfm/lastfm_train.npy' % dataset_prefix, 
#                   'lastfm_train.npy')

In [4]:
import numpy as np

In [5]:
import tensorflow as tf
import recsys.recommenders.recommender_base as recommender_base
import recsys.modules.extractions.embedding_layer as embedding_layer
import recsys.modules.interactions.rnn_softmax as rnn_softmax

In [6]:
import imp
recommender_base = imp.reload(recommender_base)
embedding_layer = imp.reload(embedding_layer)
rnn_softmax = imp.reload(rnn_softmax)

In [7]:
def VanillaRnnRec(batch_size, dim_item_embed, max_seq_len, total_items, num_units,
        l2_reg_embed=None, init_model_dir=None,
        save_model_dir='VanillaRnnRec/', train=True, serve=False):
    
    rec = recommender_base.Recommender(init_model_dir=init_model_dir,
                      save_model_dir=save_model_dir, train=train, serve=serve)
    
    @rec.traingraph.inputgraph(outs=['seq_item_id', 'seq_len', 'label'])
    def train_input_graph(subgraph):
        subgraph['seq_item_id'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size, max_seq_len],
                                      name='seq_item_id')
        subgraph['seq_len'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size], 
                                      name='seq_len')
        subgraph['label'] = tf.placeholder(tf.int32, 
                                      shape=[batch_size], 
                                      name='label')
        subgraph.register_global_input_mapping({'seq_item_id': subgraph['seq_item_id'],
                                                'seq_len': subgraph['seq_len'],
                                                'label': subgraph['label']})
        
    @rec.servegraph.inputgraph(outs=['seq_item_id', 'seq_len'])
    def serve_input_graph(subgraph):
        subgraph['seq_item_id'] = tf.placeholder(tf.int32, 
                                      shape=[None, max_seq_len],
                                      name='seq_item_id')
        subgraph['seq_len'] = tf.placeholder(tf.int32, 
                                      shape=[None],
                                      name='seq_len')
        subgraph.register_global_input_mapping({'seq_item_id': subgraph['seq_item_id'],
                                                'seq_len': subgraph['seq_len']})
    
    @rec.traingraph.itemgraph(ins=['seq_item_id'], outs=['seq_vec'])
    @rec.servegraph.itemgraph(ins=['seq_item_id'], outs=['seq_vec'])
    def item_graph(subgraph):
        _, subgraph['seq_vec']= embedding_layer.apply(l2_reg=l2_reg_embed,
                                      init='normal',
                                      id_=subgraph['seq_item_id'],
                                      shape=[total_items, dim_item_embed],
                                      subgraph=subgraph,
                                      scope='item')

    @rec.traingraph.interactiongraph(ins=['seq_vec', 'seq_len', 'label'])
    def train_interaction_graph(subgraph):
        rnn_softmax.apply(
            sequence=subgraph['seq_vec'], 
            seq_len=subgraph['seq_len'], 
            num_units=num_units, 
            cell_type='gru',
            total_items=total_items, 
            label=subgraph['label'], 
            train=True, 
            subgraph=subgraph, 
            scope='RNNSoftmax')

    @rec.servegraph.interactiongraph(ins=['seq_vec', 'seq_len'])
    def serve_interaction_graph(subgraph):
        rnn_softmax.apply(
            sequence=subgraph['seq_vec'], 
            seq_len=subgraph['seq_len'],
            num_units=num_units,
            cell_type='gru',
            total_items=total_items, 
            train=False, 
            subgraph=subgraph, 
            scope='RNNSoftmax')

    @rec.traingraph.optimizergraph
    def optimizer_graph(subgraph):
        losses = tf.add_n(subgraph.get_global_losses())
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        subgraph.register_global_operation(optimizer.minimize(losses))

    @rec.traingraph.connector
    @rec.servegraph.connector
    def connect(graph):
        graph.itemgraph['seq_item_id'] = graph.inputgraph['seq_item_id']
        graph.interactiongraph['seq_len'] = graph.inputgraph['seq_len']
        graph.interactiongraph['seq_vec'] = graph.itemgraph['seq_vec']

    @rec.traingraph.connector.extend
    def train_connect(graph):
        graph.interactiongraph['label'] = graph.inputgraph['label']

    return rec

In [8]:
# dataset
import recsys.dataset as dataset
train_data = np.load('lastfm_train.npy')
test_data = np.load('lastfm_test.npy')
total_users = max(set(list(train_data['user_id']) + list(test_data['user_id']))) + 1
total_items = max(set(list(train_data['item_id']) + list(test_data['item_id']))) + 1
print(total_users, total_items)
train_data[:2], test_data[:2]

992 14598


(array([(0, 1304, 1241478537), (0, 1036, 1241445250)],
       dtype=[('user_id', '<i4'), ('item_id', '<i4'), ('ts', '<i4')]),
 array([(1, 282, 1240944095), (1, 282, 1240943945)],
       dtype=[('user_id', '<i4'), ('item_id', '<i4'), ('ts', '<i4')]))

In [9]:
# datasets
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(train_data, total_users, total_items, 
                        sortby='ts', name='Train')
test_dataset = dataset.Dataset(test_data, total_users, total_items, 
                       sortby='ts', name='Test')

In [10]:
# hyperparamerters
dim_item_embed = 50     # dimension of item embedding
max_seq_len = 100       # the maxium length of user's listen history
num_units = 32          # Number of units in the RNN model
total_iter = int(1e3)   # iterations for training 
batch_size = 100        # training batch size
eval_iter = 200         # iteration of evaluation
save_iter = eval_iter   # iteration of saving model   

In [11]:
# model
model = VanillaRnnRec(batch_size=batch_size, 
    dim_item_embed=dim_item_embed, 
    max_seq_len=max_seq_len, 
    total_items=train_dataset.total_items(), 
    num_units=num_units, 
    save_model_dir='VanillaRnnRec/', 
    train=True, serve=True)

In [12]:
# evaluators
import recsys.metrics.auc as auc
import recsys.metrics.ndcg as ndcg
import recsys.metrics.recall as recall
import recsys.metrics.precision as precision

auc_evaluator = auc.AUC()
ndcg_evaluator = ndcg.NDCG(ndcg_at=[100])
recall_evaluator = recall.Recall(recall_at=[100, 200, 300, 400, 500])
precision_evaluator = precision.Precision(precision_at=[100])

In [13]:
# sampler
import recsys.samplers.temporal_sampler as temporal_sampler
train_sampler = temporal_sampler.create_training_sampler(batch_size=batch_size, max_seq_len=max_seq_len, 
                                dataset=train_dataset, num_process=1)
test_sampler = temporal_sampler.create_evaluation_sampler(dataset=test_dataset, 
                                         max_seq_len=max_seq_len)

In [14]:
# trainer
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)

In [15]:
# train/test
trainer.train(total_iter=total_iter, 
    eval_iter=eval_iter,
    save_iter=save_iter,
    train_sampler=train_sampler,
    eval_samplers=[test_sampler], 
    evaluators=[auc_evaluator, ndcg_evaluator, recall_evaluator, precision_evaluator])

[Training starts, total_iter: 1000, eval_iter: 200, save_iter: 200]
[iter 200] Model saved.
[iter 200] loss: 9.145113
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from VanillaRnnRec/model.ckpt
..(dataset: Test) AUC 0.727135027745427
..(dataset: Test) NDCG 0.013805037637582434
..(dataset: Test) Recall 0.056666666666666664 0.11 0.13666666666666666 0.18 0.19
..(dataset: Test) Precision 0.0005666666666666667
[iter 400] Model saved.
[iter 400] loss: 8.827863
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from VanillaRnnRec/model.ckpt
..(dataset: Test) AUC 0.7564718777831061
..(dataset: Test) NDCG 0.012652879796065783
..(dataset: Test) Recall 0.06666666666666667 0.12333333333333334 0.16333333333333333 0.19 0.24
..(dataset: Test) Precision 0.0006666666666666668
[iter 600] Model saved.
[iter 600] loss: 8.742294
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from VanillaRnnRec/model.ckpt
..(dataset: Test) AUC 0.7569059852481104
..(datas